In [14]:
# Import necessary libraries
from IPython.display import HTML, display, clear_output
import ipywidgets as widgets
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files
import io

# Custom CSS styling for the UI
HTML("""
<style>
  .widget-button {
    background: linear-gradient(45deg, #4CAF50, #45a049) !important;
    border-radius: 20px !important;
    color: white !important;
    font-weight: bold !important;
    padding: 10px 20px !important;
    margin: 5px !important;
    border: none !important;
    transition: transform 0.3s !important;
  }
  .widget-button:hover {
    transform: scale(1.05);
    box-shadow: 0 4px 8px rgba(0,0,0,0.2);
  }
  .header {
    color: #4CAF50;
    font-family: 'Arial';
    text-align: center;
    font-size: 2.5em;
    text-shadow: 2px 2px 4px rgba(0,0,0,0.2);
    margin: 20px 0;
  }
  .output_area {
    border-radius: 15px;
    padding: 20px;
    margin: 10px 0;
    background: #f8f9fa;
    box-shadow: 0 4px 8px rgba(0,0,0,0.1);
  }
  .error-message {
    color: red;
    font-weight: bold;
  }
  .success-message {
    color: #4CAF50;
    font-weight: bold;
  }
</style>
""")

# Create UI elements
header = widgets.HTML("<h1 class='header'>🔐 Secret Image Vault</h1>")
upload_cover = widgets.FileUpload(description="Upload Cover Image:", multiple=False)
upload_secret = widgets.FileUpload(description="Upload Secret Image:", multiple=False)
process_btn = widgets.Button(description="🚀 Encode Images", button_style='success')
show_encoded_btn = widgets.Button(description="Show Encoded Image", button_style='primary')
show_hidden_btn = widgets.Button(description="Reveal Hidden Image", button_style='warning')
output = widgets.Output()
image_display = widgets.Output()

# Layout adjustments
upload_cover.layout.width = '300px'
upload_secret.layout.width = '300px'
buttons_box = widgets.HBox([show_encoded_btn, show_hidden_btn])
main_box = widgets.VBox([
    header,
    upload_cover,
    upload_secret,
    process_btn,
    buttons_box,
    image_display
], layout=widgets.Layout(align_items='center'))

# Global variables
encoded_image = None
decoded_image = None

# Image encoding function
def encode_images(b):
    global encoded_image, decoded_image
    with output:
        clear_output()

        try:
            # Check if both images are uploaded
            if not upload_cover.value or not upload_secret.value:
                raise ValueError("Please upload both cover and secret images!")

            # Get uploaded images
            cover_data = next(iter(upload_cover.value.values()))['content']
            secret_data = next(iter(upload_secret.value.values()))['content']

            # Open and convert images
            cover = Image.open(io.BytesIO(cover_data)).convert("RGB")
            secret = Image.open(io.BytesIO(secret_data)).convert("RGB")

            # Resize secret to match cover dimensions
            secret = secret.resize(cover.size)

            # Convert to numpy arrays
            cover_arr = np.array(cover)
            secret_arr = np.array(secret)

            # Steganography process
            cover_arr &= 0b11110000  # Clear LSBs of cover image
            secret_bits = (secret_arr >> 4).astype(np.uint8)  # Get MSBs of secret image
            encoded_arr = cover_arr | secret_bits  # Combine images

            # Save results
            encoded_image = Image.fromarray(encoded_arr)
            decoded_image = decode_image(encoded_image)

            display(HTML("<h3 class='success-message'>✅ Encoding Successful!</h3>"))
            add_download_buttons()

        except Exception as e:
            display(HTML(f"<h3 class='error-message'>❌ Error: {str(e)}</h3>"))

# Image decoding function
def decode_image(encoded_img):
    encoded_arr = np.array(encoded_img)
    secret_bits = (encoded_arr & 0b00001111).astype(np.uint8)  # Extract LSBs
    return Image.fromarray(secret_bits << 4)  # Shift to MSBs

# Function to display encoded or hidden images
def show_encoded(b):
    with image_display:
        clear_output()
        if encoded_image:
            plt.figure(figsize=(8, 6))
            plt.imshow(encoded_image)
            plt.title("Encoded Image", fontsize=14, pad=20, color='#4CAF50')
            plt.axis('off')
            plt.show()
        else:
            display(HTML("<p class='error-message'>⚠️ Encode images first!</p>"))

def show_hidden(b):
    with image_display:
        clear_output()
        if decoded_image:
            plt.figure(figsize=(8, 6))
            plt.imshow(decoded_image)
            plt.title("Revealed Secret", fontsize=14, pad=20, color='#FF5722')
            plt.axis('off')
            plt.show()
        else:
            display(HTML("<p class='error-message'>⚠️ Encode images first!</p>"))

# Function to add download buttons
def add_download_buttons():
    with image_display:
        clear_output()

        # Create download buttons
        encoded_download_btn = widgets.Button(
            description="📥 Download Encoded Image",
            button_style='info',
            layout=widgets.Layout(width='250px')
        )

        secret_download_btn = widgets.Button(
            description="📥 Download Secret Image",
            button_style='danger',
            layout=widgets.Layout(width='250px')
        )

        # Define button click handlers
        def on_encoded_download_click(b):
            encoded_image.save("encoded_image.png")
            files.download("encoded_image.png")

        def on_secret_download_click(b):
            decoded_image.save("decoded_secret.png")
            files.download("decoded_secret.png")

        # Attach handlers to buttons
        encoded_download_btn.on_click(on_encoded_download_click)
        secret_download_btn.on_click(on_secret_download_click)

        # Display buttons
        display(widgets.HBox([encoded_download_btn, secret_download_btn]))

# Event handlers
process_btn.on_click(encode_images)
show_encoded_btn.on_click(show_encoded)
show_hidden_btn.on_click(show_hidden)

# Display the UI
display(main_box, output)

Output()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>